In [1]:
import scipy.linalg as la
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
import numpy as np
from helper_functions.grape_functions import *
from main_grape.Grape import Grape
%pylab inline



Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [8]:
def run_QFT(qubit_num):


    #Defining time scales
    total_time = qubit_num *2.0
    steps = qubit_num * 120



    state_transfer = False
    use_inter_vecs = True
    #Defining H0
    qubit_state_num = 2
    freq_ge = 0#GHz
    g = 2*np.pi*0.5 #GHz

    ens = np.array([ 2*np.pi*ii*(freq_ge) for ii in np.arange(qubit_state_num)])
    Q_x   = np.diag(np.sqrt(np.arange(1,qubit_state_num)),1)+np.diag(np.sqrt(np.arange(1,qubit_state_num)),-1)
    Q_y   = (0+1j) *(np.diag(np.sqrt(np.arange(1,qubit_state_num)),1)-np.diag(np.sqrt(np.arange(1,qubit_state_num)),-1))
    Q_z   = np.diag(np.arange(0,qubit_state_num))
    I_q = np.identity(qubit_state_num)
    H_q = np.diag(ens)

    X = multi_kron(Q_x,qubit_num)
    Z = multi_kron(Q_z,qubit_num)
    Y = multi_kron(Q_y,qubit_num)

    g_op = nn_chain_kron(Q_x, I_q, qubit_num, qubit_state_num)


    H0 = kron_all(H_q,qubit_num, I_q) + g * g_op

    # Defining dressed info
    is_dressed = False
#     w_c, v_c, dressed_id = get_dressed_info(H0)
#     dressed_info = {'dressed_id':dressed_id, 'eigenvectors':v_c, 'eigenvalues':w_c,'is_dressed':is_dressed}
    
    dressed_info = None


    #Defining Concerned states (starting states)

    psi0 = concerned(qubit_num,qubit_state_num)


    #Defining states to include in the drawing of occupation
    states_draw_list = range(qubit_state_num**qubit_num)
    states_draw_names =[]
    for ii in states_draw_list:
        states_draw_names.append(Basis(ii,qubit_num,qubit_state_num))

    #Defining U (Target)

    if is_dressed:    
        U = dressed_unitary(transmon_gate(qft(qubit_num),qubit_state_num),v_c,dressed_id)
    else:   
        U= transmon_gate(qft(qubit_num),qubit_state_num)

    if state_transfer:
        target_vec_list = []

        for ii in psi0:
            target_vec = np.dot(U,v_c[:,get_state_index(ii,dressed_id)])
            target_vec_list.append(target_vec)

        U = target_vec_list


    # print target_vec_list[1]
    # print np.inner(np.conjugate(target_vec_list[0]),target_vec_list[1])

    #Defining U0 (Initial)
    q_identity = np.identity(qubit_state_num**qubit_num)
    U0= q_identity

    #Defining control Hs

    Hops = []
    Hnames = []
    ops_max_amp = []
    max_amp=3.0
    Hops,Hnames,ops_max_amp = append_separate_krons(Q_x,'x',qubit_num,qubit_state_num,Hops,Hnames,ops_max_amp,amp=max_amp)
    Hops,Hnames,ops_max_amp = append_separate_krons(Q_z,'z',qubit_num,qubit_state_num,Hops,Hnames,ops_max_amp,amp=max_amp)

    print Hnames

    #Defining convergence parameters
    max_iterations = 10000
    decay = 10000 #max_iterations/2
    convergence = {'rate':0.1, 'update_step':10 ,'max_iterations':max_iterations,\
                   'conv_target':1e-3,'learning_rate_decay':decay,'min_grad':1e-60}

    reg_coeffs = {'envelope':0,'dwdt':0,'d2wdt2':0}
    
#     import h5py

#     with h5py.File('/home/nelson/Simulations/GRAPE-Data/spin_chain_qft/00000_gpu_spin_chain_qft_N3.h5','r') as hf:
#         u0 = np.array(hf.get('uks'))[-1]
        
    print U
    
    u0=None

    uks,U_final = Grape(H0,Hops,Hnames,U,total_time,steps,psi0,convergence=convergence, draw = [states_draw_list,states_draw_names],  

                        show_plots = False, use_gpu = False,state_transfer = state_transfer, use_inter_vecs = use_inter_vecs,
           unitary_error = 1e-8,  maxA=ops_max_amp,Taylor_terms =[14,0],initial_guess=u0,
                       dressed_info = dressed_info, method = 'ADAM', reg_coeffs=reg_coeffs, file_name='gpu_spin_chain_qft_N%d' %(qubit_num), 
                        data_path = '/home/nelson/Simulations/GRAPE-Data/spin_chain_qft')

In [11]:
run_QFT(qubit_num=3)
#run_QFT(qubit_num=9)

['xii', 'ixi', 'iix', 'zii', 'izi', 'iiz']
[[  3.53553391e-01 +0.00000000e+00j   3.53553391e-01 +0.00000000e+00j
    3.53553391e-01 +0.00000000e+00j   3.53553391e-01 +0.00000000e+00j
    3.53553391e-01 +0.00000000e+00j   3.53553391e-01 +0.00000000e+00j
    3.53553391e-01 +0.00000000e+00j   3.53553391e-01 +0.00000000e+00j]
 [  3.53553391e-01 +0.00000000e+00j   2.50000000e-01 +2.50000000e-01j
    2.16489014e-17 +3.53553391e-01j  -2.50000000e-01 +2.50000000e-01j
   -3.53553391e-01 +4.32978028e-17j  -2.50000000e-01 -2.50000000e-01j
   -6.49467042e-17 -3.53553391e-01j   2.50000000e-01 -2.50000000e-01j]
 [  3.53553391e-01 +0.00000000e+00j   2.16489014e-17 +3.53553391e-01j
   -3.53553391e-01 +4.32978028e-17j  -6.49467042e-17 -3.53553391e-01j
    3.53553391e-01 -8.65956056e-17j   1.08244507e-16 +3.53553391e-01j
   -3.53553391e-01 +1.29893408e-16j  -1.51542310e-16 -3.53553391e-01j]
 [  3.53553391e-01 +0.00000000e+00j  -2.50000000e-01 +2.50000000e-01j
   -6.49467042e-17 -3.53553391e-01j   2.5000